# Evaluación Módulo 3 : Programa lealtad de una aerolínea

## 1.- EDA:Exploración inicial de los datos

In [ ]:
# En primer lugar, importamos las librerías que nos harán falta:

# Para tratamiento de datos

import pandas as pd 
import numpy as np 

# Procesos de imputación

from sklearn.impute import SimpleImputer
from sklearn.experimental import enable_iterative_imputer
from sklearn.impute import IterativeImputer
from sklearn.impute import KNNImputer

# Visuaicación de resultados

import matplotlib.pyplot as plt
import seaborn as sns
pd.set_option('display.max_columns', None) 

In [ ]:
# Carga de csv:

df_flight = pd.read_csv("files/precios_productos.csv", index_col=0)

df_flight.head()